# 利用函数注解方式实现重载

问题： 你已经学过怎样使用函数参数注解，那么你可能会想利用它来实现基于类型的方法重载。
但是你不确定应该怎样去实现（或者到底行得通不）


In [71]:
import inspect
import types

class MultiMethod:
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name
        
    def register(self, meth):
        sig = inspect.signature(meth)
        
        types = []
        for name, param in sig.parameters.items():
            if name == "self":
                continue
            if param.annotation is inspect.Parameter.empty:
                raise TypeError(
                    'Argument {} must be annotated with a type'.format(name)
                )
            if not isinstance(param.annotation, type):
                'Argument {} annotation must be a type'.format(name)
                
            #有默认参数的参数是可以省略传值的
            if param.default is not inspect.Parameter.empty:
                self._methods[tuple(types)] = meth
            types.append(param.annotation)
        self._methods[tuple(types)] = meth
    
    def __call__(self, *args):
        types = tuple(type(arg) for arg in args[1:])
        
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            raise TypeError('No matching method for types {}'.format(types))
        
    def __get__(self, instance, cls):
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self

class MultiDict(dict):
    def __setitem__(self, key, value):
        
        if key in self:
            current_value = self[key]
            
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)                
        else:
            super().__setitem__(key, value)
            
class MultipleMeta(type):
    
    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()
        
        

In [72]:
#example

class Spam(metaclass=MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)
    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)
# Example: overloaded __init__
import time
class Date(metaclass=MultipleMeta):
    def __init__(self, year: int, month:int, day:int):
        self.year = year
        self.month = month
        self.day = day
    def __init__(self):
        t = time.localtime()
        self.__init__(t.tm_year, t.tm_mon, t.tm_mday)

In [73]:
s = Spam()

In [74]:
s.bar(2,3)

Bar 1: 2 3


In [75]:
s.bar("hello", 5)

Bar 2: hello 5


In [76]:
s.bar(3, "hello")

TypeError: No matching method for types (<class 'int'>, <class 'str'>)

### <font color="red">上述的实现还有一些限制，其中一个是它不能使用关键字参数<font/>

In [77]:
s.bar(x=2,y=3)

TypeError: __call__() got an unexpected keyword argument 'x'

### <font color='red'>对于继承也是有限制的<font/>

In [81]:
class A:
    pass
class B(A):
    pass
class C:
    pass
class Spam(metaclass=MultipleMeta):
    def foo(self, x:A):
        print('Foo 1:', x)
    def foo(self, x:C):
        print('Foo 2:', x)


In [82]:
s = Spam()

In [83]:
a = A()

In [84]:
s.foo(a)

Foo 1: <__main__.A object at 0x7fbb64395b00>


In [88]:
c = C()
s.foo(c)

Foo 2: <__main__.C object at 0x7fbb64377908>


In [86]:
b = B()

In [87]:
s.foo(b)

TypeError: No matching method for types (<class '__main__.B'>,)